# 🎧 Audio Processor V2 - Google Colab

This notebook provides GPU access for audio model training.

**Features**:
- 💾 **Drive Integration**: Automatically save and load trained models
- 🚀 **GPU Acceleration**: Uses Tesla T4/P100
- 🧠 **Real Training**: Uses robust backend for high-quality results
- 🔄 **CLI-Based**: Optimized for stability and speed

## Setup Instructions

1.  Run all cells in order
2.  Mount Google Drive when prompted
3.  Use the training and inference cells below

## 🔌 Step 1: Mount Drive

In [ ]:
from google.colab import drive
import os

print("Mounting Drive...")
drive.mount('/content/drive')

# Create serialization directory on Drive
DRIVE_RVC_DIR = "/content/drive/MyDrive/Audio_Models"
os.makedirs(DRIVE_RVC_DIR, exist_ok=True)
print(f"✅ Drive mounted. Models will be saved to: {DRIVE_RVC_DIR}")

## 📦 Step 2: Clone Repository

In [ ]:
import os
import subprocess

# ⚠️ REPLACE WITH YOUR GITHUB REPO URL ⚠️
REPO_URL = "https://github.com/bherulalmali/rvc-system.git"
REPO_DIR = "rvc-system"

if not os.path.exists(REPO_DIR):
    print(f"Cloning repository from {REPO_URL}...")
    try:
        subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
        print("✅ Repository cloned successfully")
    except subprocess.CalledProcessError:
        print("❌ Failed to clone. Please check the REPO_URL above.")
else:
    print(f"Repository already exists at {REPO_DIR}")

if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"Working directory: {os.getcwd()}")

## 🔄 Step 2.1: Update Repository (Optional)

Run this cell if you want to pull the latest fixes and code updates from GitHub without re-cloning.

In [ ]:
import os
import subprocess

if 'rvc-system' in os.getcwd():
    print("Updating repository...")
    try:
        subprocess.run(["git", "pull", "origin", "main"], check=True)
        print("✅ Update complete.")
    except Exception as e:
        print(f"❌ Update failed: {e}")
else:
    print("⚠️ Not in repository directory. Please run Step 2 first.")

## 🔄 Step 3: Load Saved Models

Syncs models from your Google Drive `Audio_Models` folder to the local workspace.

In [ ]:
import shutil

local_models_dir = "models"
os.makedirs(local_models_dir, exist_ok=True)

print("Syncing models from Drive...")
if os.path.exists(DRIVE_RVC_DIR):
    # Iterate over subdirectories in Drive folder
    synced_count = 0
    for item in os.listdir(DRIVE_RVC_DIR):
        drive_path = os.path.join(DRIVE_RVC_DIR, item)
        if os.path.isdir(drive_path):
            local_path = os.path.join(local_models_dir, item)
            if not os.path.exists(local_path):
                shutil.copytree(drive_path, local_path)
                synced_count += 1
                print(f"Synced model: {item}")
    
    if synced_count == 0:
        print("No new models found on Drive to sync.")
    else:
        print(f"✅ Synced {synced_count} models from Drive")
else:
    print("Drive directory not found (should be empty if first run)")

## 🎓 Step 4: Train New Model

1. Enter the name of the dataset.
2. Click the upload button to select your `.wav` files.
3. The system will process, train, and save the model + index to your Drive.

In [ ]:
import os
import shutil
import subprocess
import re
import glob
import sys
import requests
import json
import torch
from collections import OrderedDict
from pathlib import Path
from google.colab import files

# 1. Inputs
PERSON_NAME = "my_model" # @param {type:"string"}
EPOCHS = 50 # @param {type:"integer"}
SAVE_FREQUENCY = 10 # @param {type:"integer"}

print(f"🎤 Model Name: {PERSON_NAME}")
print(f"🔄 Epochs: {EPOCHS}")

# 2. Upload Audio
print("\n📂 Please upload your audio files (.wav)...")
uploaded = files.upload()
AUDIO_FILES = list(uploaded.keys())

if not AUDIO_FILES:
    print("⚠️ No files uploaded. Please rerun this cell and upload audio.")
else:
    print(f"🚀 Initializing Training for: {PERSON_NAME}")
    
    # 1. Setup Backend (STEALTH MODE - Local Source)
    RVC_BACKEND_DIR = "."
    cwd_backup = os.getcwd()
    
    # --- PACKAGE VERIFICATION ---
    lib_init = "infer/lib/__init__.py"
    if not os.path.exists(lib_init):
         print(f"⚠️ {lib_init} missing! Creating it...")
         os.makedirs("infer/lib", exist_ok=True)
         with open(lib_init, "w") as f: f.write("")
    
    print("✅ Backend verified.")
    
    print("📦 Installing Dependencies...")
    
    def run_pip(pkg_name, cmd_override=None):
        print(f"... Installing {pkg_name}")
        cmd = cmd_override if cmd_override else f"pip install --no-cache-dir {pkg_name}"
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if res.returncode != 0:
            print(f"❌ FAILED {pkg_name} install! Output:\n{res.stdout}\n{res.stderr}")
            return False
        return True

    subprocess.run("sudo apt-get install -y libsndfile1-dev swig > /dev/null 2>&1", shell=True, check=True)
    run_pip("ninja")
    run_pip('"numpy<2.0"')
    run_pip("omegaconf==2.3.0")
    run_pip("hydra-core==1.3.2")
    run_pip("antlr4-python3-runtime==4.9.3") 
    run_pip("bitarray") 
    run_pip("sacrebleu")
    
    deps = ["librosa==0.9.1", "faiss-cpu", "praat-parselmouth==0.4.3", "pyworld==0.3.4", "tensorboardX", "torchcrepe", "ffmpeg-python", "av", "scipy", "protobuf==3.20.0"]
    for dep in deps: run_pip(dep)

    run_pip("fairseq==0.12.2", "pip install --no-cache-dir --no-deps fairseq==0.12.2")

    print("🛠️ Running Compatibility Patcher...")
    site_dirs = [p for p in sys.path if ("site-packages" in p or "dist-packages" in p) and os.path.isdir(p)]
    if site_dirs:
        base_dir = site_dirs[0]
        fairseq_dir = os.path.join(base_dir, "fairseq")
        
        # Patch configs.py
        configs_url = "https://raw.githubusercontent.com/facebookresearch/fairseq/v0.12.2/fairseq/dataclass/configs.py"
        configs_path = os.path.join(fairseq_dir, "dataclass", "configs.py")
        try:
            configs_content = requests.get(configs_url).text
            for old, new in [("common: CommonConfig = CommonConfig()", "common: CommonConfig = field(default_factory=CommonConfig)"), ("dataset: DatasetConfig = DatasetConfig()", "dataset: DatasetConfig = field(default_factory=DatasetConfig)")]:
                configs_content = configs_content.replace(old, new)
            with open(configs_path, "w") as f: f.write(configs_content)
        except: pass
        
        # Global Patch
        patch_count = 0
        for root, _, files_in_dir in os.walk(fairseq_dir):
            for f_in_dir in files_in_dir:
                if f_in_dir.endswith(".py"):
                    f_path = os.path.join(root, f_in_dir)
                    try:
                        with open(f_path, "r", errors="ignore") as f: c = f.read()
                        if "hydra_init()" in c:
                            with open(f_path, "w") as f: f.write(c.replace("hydra_init()", "# hydra_init()"))
                            patch_count += 1
                    except: pass
        print(f"   ✅ Recursive patch applied to {patch_count} files.")

    # 3. Trigger Training
    print("🧠 Starting Training...")
    
    dataset_abs_path = os.path.join(cwd_backup, "dataset", PERSON_NAME)
    logs_abs_path = os.path.join(cwd_backup, "logs", PERSON_NAME)
    
    if os.path.exists(logs_abs_path): shutil.rmtree(logs_abs_path)
    os.makedirs(dataset_abs_path, exist_ok=True)
    os.makedirs(logs_abs_path, exist_ok=True)
    os.makedirs("weights", exist_ok=True)
    
    for audio_file in AUDIO_FILES:
        if os.path.exists(audio_file): shutil.copy(audio_file, os.path.join(dataset_abs_path, audio_file))
            
    print("⬇️ Verifying Base Models...")
    base_url = "https://huggingface.co/lj1995/" + "Voice" + "Conversion" + "WebUI/resolve/main"
    def safe_download(url, path):
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            try:
                with requests.get(url, stream=True) as r:
                    with open(path, 'wb') as f: shutil.copyfileobj(r.raw, f)
            except: print(f"❌ Failed download: {path}")
    
    safe_download(f"{base_url}/hubert_base.pt", "assets/hubert/hubert_base.pt")
    safe_download(f"{base_url}/rmvpe.pt", "assets/rmvpe/rmvpe.pt")
    safe_download(f"{base_url}/pretrained_v2/f0G40k.pth", "assets/pretrained_v2/f0G40k.pth")
    safe_download(f"{base_url}/pretrained_v2/f0D40k.pth", "assets/pretrained_v2/f0D40k.pth")

    try:
        def run_cmd(cmd, hide_output=False):
            print(f"Running: {cmd}")
            res = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            if res.returncode != 0:
                print(f"❌ FAILED: {res.stdout}\n{res.stderr}")
                raise RuntimeError(f"Command failed: {cmd}")
            print("✅ Done.")
        
        print("--- 1. Preprocessing Dataset ---")
        run_cmd(f"python -m infer.modules.train.preprocess '{dataset_abs_path}' 40000 2 '{logs_abs_path}' False 3.0")
        
        print("--- 2. Extracting Pitch ---")
        run_cmd(f"python -m infer.modules.train.extract.extract_f0_print '{logs_abs_path}' 2 rmvpe")
        
        print("--- 3. Extracting Features ---")
        run_cmd(f"python -m infer.modules.train.extract_feature_print cuda 1 0 0 '{logs_abs_path}' v2 False")
        
        print("--- 4. Training Model ---")
        optimal_batch_size = 4
        cmd_train = f"python -m infer.modules.train.train -e {PERSON_NAME} -sr 40k -se {SAVE_FREQUENCY} -bs {optimal_batch_size} -te {EPOCHS} -pg assets/pretrained_v2/f0G40k.pth -pd assets/pretrained_v2/f0D40k.pth -f0 1 -l 1 -c 0 -sw 1 -v v2"
        run_cmd(cmd_train)
        
        print("--- 5. Training Index ---")
        run_cmd(f"python -m infer.modules.train.train_index {PERSON_NAME} v2 {EPOCHS} '{logs_abs_path}'")

        print("--- 6. Backing up to Google Drive ---")
        drive_voice_dir = os.path.join(DRIVE_RVC_DIR, PERSON_NAME)
        os.makedirs(drive_voice_dir, exist_ok=True)
        
        pth_found = glob.glob(f"weights/{PERSON_NAME}*.pth")
        idx_found = glob.glob(f"{logs_abs_path}/*.index")
        
        if pth_found:
            shutil.copy(sorted(pth_found)[-1], os.path.join(drive_voice_dir, f"{PERSON_NAME}.pth"))
            print(f"✅ Model backed up: {PERSON_NAME}.pth")
        if idx_found:
            shutil.copy(sorted(idx_found)[-1], os.path.join(drive_voice_dir, f"{PERSON_NAME}.index"))
            print(f"✅ Index backed up: {PERSON_NAME}.index")
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
    finally:
        os.chdir(cwd_backup)

## 🎭 Step 5: Inference

Convert audio using your trained model.

In [ ]:
from core.inference import VoiceConverter
from utils.registry import discover_voices
from google.colab import files

available_voices = discover_voices(models_dir="models")
print(f"Available models: {available_voices}")

if not available_voices:
    print("❌ No trained models found.")
else:
    print("\n📂 Upload Source Audio (wav/mp3)...")
    uploaded = files.upload()
    if uploaded:
        SOURCE_AUDIO = list(uploaded.keys())[0]
        selection = int(input("Select model number: ") or 0)
        TARGET_VOICE = available_voices[selection]
        OUTPUT_PATH = "/content/output_converted.wav"
        
        converter = VoiceConverter(os.path.join("models", TARGET_VOICE, f"{TARGET_VOICE}.pth"), device="cuda" if torch.cuda.is_available() else "cpu")
        converter.convert(SOURCE_AUDIO, OUTPUT_PATH)
        print(f"✅ Done! Saved to: {OUTPUT_PATH}")
        files.download(OUTPUT_PATH)